In [1]:
import graph_tool as gt
import torch
from src.diffusion_model_discrete import DiscreteDenoisingDiffusion
import pickle
import os
import torch.distributed as dist

In [2]:
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
dist.init_process_group('gloo', rank=0, world_size=1)
def get_model_facebook():

    argpath = '/mnt/c/repo/watermark-graph-diffusion/model/facebook.pkl'
    modelpath = '/mnt/c/repo/watermark-graph-diffusion/model/facebook-epoch=3099.ckpt'
    args = pickle.load(open(argpath, 'rb'))
    model = DiscreteDenoisingDiffusion.load_from_checkpoint(modelpath, **args).to('cuda')
    model.eval()
    return model

model = get_model_facebook()


(ipykernel_launcher.py:22263): Gtk-WARNING **: 00:38:32.863: Locale not supported by C library.
	Using the fallback 'C' locale.
<frozen importlib._bootstrap>:228: RuntimeWarning: to-Python converter for std::pair<double, double> already registered; second conversion method ignored.


Marginal distribution of the classes: tensor([1.]) for nodes, tensor([0.7001, 0.2999]) for edges


/home/renyi/anaconda3/envs/digress/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [9]:
samples_left_to_generate = model.cfg.general.final_model_samples_to_generate
samples_left_to_save = model.cfg.general.final_model_samples_to_save
chains_left_to_save = model.cfg.general.final_model_chains_to_save
samples_left_to_generate = 120
samples = []
id = 0
while samples_left_to_generate > 0:
    bs = 2 * model.cfg.train.batch_size
    to_generate = min(samples_left_to_generate, bs)
    to_save = min(samples_left_to_save, bs)
    chains_save = min(chains_left_to_save, bs)
    samples.extend(model.sample_batch(id, to_generate, num_nodes=None, save_final=to_save,
                                     keep_chain=chains_save, number_chain_steps=model.number_chain_steps))
    id += to_generate
    samples_left_to_save -= to_save
    samples_left_to_generate -= to_generate
    chains_left_to_save -= chains_save

In [11]:

with open("../samples/facebook-free.pkl", "wb") as f:
     pickle.dump(samples, f)

In [10]:
from datasets.social_dataset import SocialGraphDataModule
from analysis.social_utils import SocialSamplingMetrics
from g2gcompress import Graph2GraphPairCompress
g2gc = Graph2GraphPairCompress() if model.cfg.train.get("g2gc", False) else None
datamodule = SocialGraphDataModule(model.cfg, g2gc=g2gc)
sampling_metrics = SocialSamplingMetrics(datamodule, g2gc=g2gc)

In [12]:
sampling_metrics.forward(samples, None, None, None,  local_rank=model.local_rank, test=True)

Computing sampling metrics between 120 generated graphs and 40 test graphs -- emd computation: False
Building networkx graphs...
Computing degree stats..
Computing spectre stats...
Computing clustering stats...
Computing orbit stats...
Computing all fractions...
Sampling statistics {'degree': 0.02268549309415646, 'spectre': 0.013795830147649957, 'clustering': 0.03334145346846341, 'orbit': 0.033692870125542056, 'sampling/frac_unique': 1.0, 'sampling/frac_unique_non_iso': 1.0, 'sampling/frac_unic_non_iso_valid': 0.0, 'sampling/frac_non_iso': 1.0}


In [7]:
def read_samples_from_file(filename):
    samples = []
    with open(filename, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line.startswith("N="):
                n = int(line.strip().split('=')[1])
                f.readline() 

                atoms_line = f.readline().strip()
                atoms = [float(x) for x in atoms_line.split()]
                atoms_tensor = torch.tensor(atoms).view(1, n)  

                f.readline()  

                bonds = []
                while True:
                    bond_line = f.readline().strip()
                    if bond_line == "":
                        break
                    bond_list = [int(b) for b in bond_line.split()]
                    bonds.append(bond_list)
                bonds_tensor = torch.tensor(bonds).view(n, n)  

                samples.append((atoms_tensor, bonds_tensor))
    return samples

filename = "/mnt/c/DiGress/generated_samples4.txt"
samples = read_samples_from_file(filename)
print(samples)


[(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0, 0, 0,  ..., 0, 1, 1],
        [0, 0, 0,  ..., 0, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 0, 1, 0]])), (tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), tensor([[0, 1, 1,  ..., 1, 0, 1],
        [1, 0, 1,  ..., 0, 1, 1],
        [1, 1, 0,  ..., 1, 1, 0],
        ...,
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 0, 1, 0]])), (tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 

In [8]:
sampling_metrics.forward(samples, None, None, None,  local_rank=model.local_rank, test=True)

Computing sampling metrics between 200 generated graphs and 40 test graphs -- emd computation: False
Building networkx graphs...
Computing degree stats..
Computing spectre stats...


<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: normalized_laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


Computing clustering stats...
Computing orbit stats...
Computing all fractions...
Sampling statistics {'degree': 0.009616931678670948, 'spectre': 0.009248213665478344, 'clustering': 0.029986287521690042, 'orbit': 0.02994789143539138, 'sampling/frac_unique': 1.0, 'sampling/frac_unique_non_iso': 1.0, 'sampling/frac_unic_non_iso_valid': 0.0, 'sampling/frac_non_iso': 1.0}


In [5]:
samples_batch = []
for i in range(3):
    samples , states = model.sample_batch_simplified(None, 1)
    samples_batch.append(samples)
print(samples_batch)
# dump samples
#with open("../samples/watermarked.pkl", "wb") as f:
#    pickle.dump(samples, f)

100%|██████████| 40/40 [19:54<00:00, 29.87s/it]

[[[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0]), tensor([[0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        ...,
        [1, 0, 1,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])], [tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        [0, 0, 0,  ..., 1, 1, 0],
        ...,
        [0, 1, 1,  ..., 0, 1, 0],
        [0, 1, 1,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])], [tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), tensor([[0, 0, 0,  ..., 0, 0, 1],
 

In [8]:
#samples , states = model.sample_batch_simplified(None, 40)
# dump samples
#with open("../samples/no-watermark.pkl", "wb") as f:
#    pickle.dump(samples, f)

100%|██████████| 40/40 [21:13<00:00, 31.84s/it]


In [4]:
with open("../samples/no-watermark.pkl", "rb") as f:
    free = pickle.load(f) 
    
with open("../samples/watermarked.pkl", "rb") as f:
    watermark = pickle.load(f) 

Computing sampling metrics between 40 generated graphs and 40 test graphs -- emd computation: False
Building networkx graphs...
Computing degree stats..
Computing spectre stats...


<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: normalized_laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


Computing clustering stats...
Computing orbit stats...
Computing all fractions...
Sampling statistics {'degree': 0.03192500367767703, 'spectre': 0.01552635042094419, 'clustering': 0.05000636160389097, 'orbit': 0.05032700571989024, 'sampling/frac_unique': 1.0, 'sampling/frac_unique_non_iso': 1.0, 'sampling/frac_unic_non_iso_valid': 0.0, 'sampling/frac_non_iso': 1.0}
Computing sampling metrics between 40 generated graphs and 40 test graphs -- emd computation: False
Building networkx graphs...
Computing degree stats..
Computing spectre stats...
Computing clustering stats...
Computing orbit stats...
Computing all fractions...
Sampling statistics {'degree': 0.02683034583755961, 'spectre': 0.017809635282541425, 'clustering': 0.05000855314979903, 'orbit': 0.05156472107948706, 'sampling/frac_unique': 1.0, 'sampling/frac_unique_non_iso': 1.0, 'sampling/frac_unic_non_iso_valid': 0.0, 'sampling/frac_non_iso': 1.0}
Computing sampling metrics between 40 generated graphs and 40 test graphs -- emd co

['degree', 'clustering', 'orbit', 'spectre']